<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/266_MissionOrchestratorAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Human-in-the-Loop (HITL) utilities for Mission Orchestrator Agent

In [ ]:
"""Human-in-the-Loop (HITL) utilities for Mission Orchestrator Agent"""

from typing import List, Dict, Any, Optional
from datetime import datetime


def check_task_requires_approval(task: Dict[str, Any]) -> bool:
    """
    Check if a task requires human approval.

    Args:
        task: Task dictionary with 'requires_human_approval' field

    Returns:
        True if task requires approval, False otherwise
    """
    return task.get("requires_human_approval", False)


def find_tasks_requiring_approval(executed_tasks: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Find tasks that require approval and haven't been approved yet.

    Args:
        executed_tasks: List of executed task results

    Returns:
        List of tasks that need approval
    """
    pending = []

    for task_result in executed_tasks:
        task_id = task_result.get("task_id")
        status = task_result.get("status")

        # Check if task requires approval and is completed but not yet approved
        if status == "completed" and task_result.get("requires_approval", False):
            # Check if already approved
            if not task_result.get("approved", False):
                pending.append(task_result)

    return pending


def create_approval_request(
    task_result: Dict[str, Any],
    requested_at: Optional[str] = None
) -> Dict[str, Any]:
    """
    Create an approval request for a task.

    Args:
        task_result: Task execution result
        requested_at: ISO timestamp (defaults to now)

    Returns:
        Approval request dictionary
    """
    if requested_at is None:
        requested_at = datetime.now().isoformat()

    return {
        "task_id": task_result.get("task_id"),
        "task": task_result.get("task", ""),
        "agent_name": task_result.get("agent_name", ""),
        "result": task_result.get("result"),
        "requested_at": requested_at,
        "status": "pending"
    }


def process_approval(
    approval_request: Dict[str, Any],
    decision: str,
    decided_by: Optional[str] = None,
    decided_at: Optional[str] = None
) -> Dict[str, Any]:
    """
    Process an approval decision.

    Args:
        approval_request: Approval request dictionary
        decision: "approved" or "rejected"
        decided_by: Human identifier (defaults to "human")
        decided_at: ISO timestamp (defaults to now)

    Returns:
        Approval history entry
    """
    if decided_by is None:
        decided_by = "human"

    if decided_at is None:
        decided_at = datetime.now().isoformat()

    if decision not in ["approved", "rejected"]:
        raise ValueError(f"Invalid decision: {decision}. Must be 'approved' or 'rejected'")

    return {
        "task_id": approval_request.get("task_id"),
        "decision": decision,
        "decided_at": decided_at,
        "decided_by": decided_by,
        "requested_at": approval_request.get("requested_at")
    }


def get_pending_approvals(approval_history: List[Dict[str, Any]], executed_tasks: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Get list of tasks with pending approvals.

    Args:
        approval_history: List of approval decisions
        executed_tasks: List of executed tasks

    Returns:
        List of pending approval requests
    """
    approved_task_ids = {
        entry.get("task_id") for entry in approval_history
        if entry.get("decision") == "approved"
    }

    pending = []

    for task_result in executed_tasks:
        task_id = task_result.get("task_id")

        # Task requires approval if:
        # 1. It's completed
        # 2. It requires approval (from original task definition)
        # 3. It hasn't been approved yet
        if (task_result.get("status") == "completed" and
            task_result.get("requires_approval", False) and
            task_id not in approved_task_ids):

            approval_request = create_approval_request(task_result)
            pending.append(approval_request)

    return pending


def is_task_approved(task_id: str, approval_history: List[Dict[str, Any]]) -> bool:
    """
    Check if a task has been approved.

    Args:
        task_id: Task identifier
        approval_history: List of approval decisions

    Returns:
        True if task is approved, False otherwise
    """
    for entry in approval_history:
        if entry.get("task_id") == task_id and entry.get("decision") == "approved":
            return True
    return False


def auto_approve_for_testing(
    pending_approvals: List[Dict[str, Any]],
    auto_approve: bool = True
) -> List[Dict[str, Any]]:
    """
    Auto-approve all pending approvals (for testing purposes).

    Args:
        pending_approvals: List of pending approval requests
        auto_approve: Whether to auto-approve (default: True)

    Returns:
        List of approval history entries
    """
    if not auto_approve:
        return []

    approvals = []
    for request in pending_approvals:
        approval = process_approval(request, "approved", decided_by="auto_approval")
        approvals.append(approval)

    return approvals



# Approval Check Node

In [ ]:
def approval_check_node(state: MissionOrchestratorState) -> Dict[str, Any]:
    """
    Approval Check Node: Handle human approval for tasks requiring HITL.

    This node:
    1. Identifies tasks that require approval
    2. Creates pending approval requests
    3. Handles auto-approval if configured (for testing)
    4. Updates approval history

    This is a conditional node - only executes if tasks require approval.

    Input:
        - executed_tasks (List[Dict]): Completed tasks
        - approval_history (List[Dict]): Previous approval decisions
        - mission_status (str): Current mission status

    Output:
        - pending_approvals (List[Dict]): Tasks awaiting approval
        - approval_history (List[Dict]): Updated approval history
        - mission_status (str): Updated status (may be "awaiting_approval")
        - errors (List[str]): Any errors encountered
    """
    errors = state.get("errors", [])
    executed_tasks = state.get("executed_tasks", [])
    approval_history = state.get("approval_history", [])
    mission_status = state.get("mission_status", "in_progress")
    config = MissionOrchestratorConfig()

    try:
        # Get pending approvals
        pending_approvals = get_pending_approvals(approval_history, executed_tasks)

        # If auto-approve is enabled (for testing), auto-approve all pending
        if config.auto_approve_for_testing and pending_approvals:
            auto_approvals = auto_approve_for_testing(pending_approvals, auto_approve=True)
            approval_history = approval_history + auto_approvals
            pending_approvals = []  # All approved now

        # Update mission status
        if pending_approvals:
            mission_status = "awaiting_approval"
        elif mission_status == "awaiting_approval" and not pending_approvals:
            # All approvals granted, back to in_progress
            mission_status = "in_progress"

        return {
            "pending_approvals": pending_approvals,
            "approval_history": approval_history,
            "mission_status": mission_status,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"approval_check_node: Unexpected error: {str(e)}"]
        }



# Standalone test script for HITL (Human-in-the-Loop) workflows

In [ ]:
"""Standalone test script for HITL (Human-in-the-Loop) workflows"""

from agents.mission_orchestrator.utilities.hitl import (
    check_task_requires_approval,
    find_tasks_requiring_approval,
    create_approval_request,
    process_approval,
    get_pending_approvals,
    is_task_approved,
    auto_approve_for_testing
)
from agents.mission_orchestrator.nodes import approval_check_node
from agents.mission_orchestrator.utilities.data_loading import (
    load_mission_tasks
)
from config import MissionOrchestratorState


def test_hitl_utilities():
    """Test HITL utilities"""
    print("=" * 60)
    print("Testing HITL Utilities")
    print("=" * 60)

    # Load tasks
    tasks = load_mission_tasks("M001")

    # Test 1: Check if task requires approval
    print("\n1. Checking which tasks require approval...")
    for task in tasks:
        requires = check_task_requires_approval(task)
        status = "✓ Requires approval" if requires else "  No approval needed"
        print(f"   {status}: {task['task_id']} - {task['task']}")

    # Test 2: Create approval request
    print("\n2. Creating approval request...")
    executed_task = {
        "task_id": "T2",
        "task": "Verify documents",
        "status": "completed",
        "agent_name": "Document Verification Agent",
        "result": {"output": "Documents verified"},
        "requires_approval": True
    }

    approval_request = create_approval_request(executed_task)
    print(f"   ✓ Created approval request:")
    print(f"     Task: {approval_request['task_id']} - {approval_request['task']}")
    print(f"     Agent: {approval_request['agent_name']}")
    print(f"     Status: {approval_request['status']}")

    # Test 3: Process approval
    print("\n3. Processing approval decision...")
    approval_entry = process_approval(approval_request, "approved", decided_by="test_user")
    print(f"   ✓ Approval processed:")
    print(f"     Task: {approval_entry['task_id']}")
    print(f"     Decision: {approval_entry['decision']}")
    print(f"     Decided by: {approval_entry['decided_by']}")

    # Test 4: Check if task is approved
    print("\n4. Checking approval status...")
    approval_history = [approval_entry]
    is_approved = is_task_approved("T2", approval_history)
    print(f"   ✓ Task T2 approved: {is_approved}")

    is_approved = is_task_approved("T1", approval_history)
    print(f"   ✓ Task T1 approved: {is_approved} (not in history)")

    # Test 5: Get pending approvals
    print("\n5. Getting pending approvals...")
    executed_tasks = [
        {
            "task_id": "T1",
            "status": "completed",
            "requires_approval": False
        },
        {
            "task_id": "T2",
            "status": "completed",
            "requires_approval": True
        },
        {
            "task_id": "T3",
            "status": "completed",
            "requires_approval": False
        }
    ]

    pending = get_pending_approvals(approval_history, executed_tasks)
    print(f"   ✓ Found {len(pending)} pending approvals")
    for req in pending:
        print(f"     - {req['task_id']}: {req['task']}")

    # Test 6: Auto-approve for testing
    print("\n6. Auto-approving for testing...")
    auto_approvals = auto_approve_for_testing(pending, auto_approve=True)
    print(f"   ✓ Auto-approved {len(auto_approvals)} tasks")
    for approval in auto_approvals:
        print(f"     - {approval['task_id']}: {approval['decision']} by {approval['decided_by']}")

    print("\n" + "=" * 60)
    print("HITL Utilities Tests Complete!")
    print("=" * 60)


def test_approval_check_node():
    """Test approval check node"""
    print("\n" + "=" * 60)
    print("Testing Approval Check Node")
    print("=" * 60)

    # Setup state with tasks requiring approval
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "executed_tasks": [
            {
                "task_id": "T1",
                "task": "Collect customer information",
                "status": "completed",
                "requires_approval": False,
                "agent_name": "Data Collection Agent"
            },
            {
                "task_id": "T2",
                "task": "Verify documents",
                "status": "completed",
                "requires_approval": True,
                "agent_name": "Document Verification Agent"
            }
        ],
        "approval_history": [],
        "mission_status": "in_progress",
        "errors": []
    }

    # Test without auto-approval (default config)
    print("\n1. Testing with default config (auto_approve_for_testing=False)...")
    # Note: The node uses config.auto_approve_for_testing which defaults to False
    result = approval_check_node(state)

    if "pending_approvals" in result:
        print(f"   ✓ Node executed successfully")
        print(f"   - Pending approvals: {len(result['pending_approvals'])}")
        print(f"   - Mission status: {result['mission_status']}")
        if result['pending_approvals']:
            for req in result['pending_approvals']:
                print(f"     ⚠ {req['task_id']}: {req['task']} (requires approval)")
        else:
            print(f"     (No pending approvals - all tasks auto-approved or don't need approval)")

    # Test with auto-approval by updating state to simulate auto-approval
    print("\n2. Testing with auto-approval (simulated)...")
    # Simulate auto-approval by manually adding approvals
    from agents.mission_orchestrator.utilities.hitl import auto_approve_for_testing
    pending = result.get('pending_approvals', [])
    if pending:
        auto_approvals = auto_approve_for_testing(pending, auto_approve=True)
        state['approval_history'] = state.get('approval_history', []) + auto_approvals
        state['pending_approvals'] = []

        result2 = approval_check_node(state)
        print(f"   ✓ After auto-approval:")
        print(f"     - Pending approvals: {len(result2.get('pending_approvals', []))}")
        print(f"     - Approval history: {len(result2.get('approval_history', []))} entries")
        print(f"     - Mission status: {result2.get('mission_status', 'unknown')}")
        for approval in result2.get('approval_history', []):
            print(f"       ✓ {approval['task_id']}: {approval['decision']} by {approval['decided_by']}")

    print("\n" + "=" * 60)
    print("Approval Check Node Test Complete!")
    print("=" * 60)


def test_full_flow_with_hitl():
    """Test full flow with HITL approval workflow"""
    print("\n" + "=" * 60)
    print("Testing Full Flow with HITL Approval Workflow")
    print("=" * 60)

    from agents.mission_orchestrator.nodes import (
        goal_node, planning_node, data_loading_node,
        task_ordering_node, task_execution_node, approval_check_node
    )

    # Start with just mission_id
    state: MissionOrchestratorState = {
        "mission_id": "M001",
        "errors": []
    }

    # Step 1-4: Goal → Planning → Data Loading → Task Ordering
    print("\n1-4. Setting up mission (goal → planning → data → ordering)...")
    goal_result = goal_node(state)
    state = {**state, **goal_result}

    planning_result = planning_node(state)
    state = {**state, **planning_result}

    data_result = data_loading_node(state)
    state = {**state, **data_result}

    ordering_result = task_ordering_node(state)
    state = {**state, **ordering_result}

    # Step 5: Execute tasks (with approval checks)
    print("\n5. Executing tasks with approval workflow...")
    max_iterations = 10
    iteration = 0

    while state.get("task_queue") and iteration < max_iterations:
        iteration += 1

        # Execute task
        execution_result = task_execution_node(state)
        state = {**state, **execution_result}

        if execution_result.get("executed_tasks"):
            last_executed = execution_result["executed_tasks"][-1]
            requires_approval = last_executed.get("requires_approval", False)
            approval_status = " (requires approval)" if requires_approval else ""
            print(f"   Task {last_executed['task_id']} completed{approval_status}")

        # Check for approvals after each task
        approval_result = approval_check_node(state)
        state = {**state, **approval_result}

        if approval_result.get("pending_approvals"):
            print(f"     ⚠ {len(approval_result['pending_approvals'])} task(s) awaiting approval")
            for req in approval_result['pending_approvals']:
                print(f"       - {req['task_id']}: {req['task']}")

        if approval_result.get("approval_history"):
            print(f"     ✓ {len(approval_result['approval_history'])} approval(s) granted")

        if not state.get("task_queue") and not state.get("pending_approvals"):
            break

    # Final status
    print(f"\n--- Mission Execution Summary ---")
    print(f"  Mission: {state['mission']['mission_name']}")
    print(f"  Tasks completed: {state['tasks_completed']}/{state['tasks_total']}")
    print(f"  Mission status: {state['mission_status']}")
    print(f"  Pending approvals: {len(state.get('pending_approvals', []))}")
    print(f"  Approval history: {len(state.get('approval_history', []))} entries")

    if state.get("approval_history"):
        print(f"\n  Approval History:")
        for approval in state['approval_history']:
            print(f"    ✓ {approval['task_id']}: {approval['decision']} by {approval['decided_by']}")

    if state.get("pending_approvals"):
        print(f"\n  Pending Approvals:")
        for req in state['pending_approvals']:
            print(f"    ⚠ {req['task_id']}: {req['task']}")

    print("\n" + "=" * 60)
    print("Full Flow with HITL Test Complete!")
    print("=" * 60)


if __name__ == "__main__":
    try:
        test_hitl_utilities()
        test_approval_check_node()
        test_full_flow_with_hitl()
        print("\n✅ All tests completed successfully!")
    except Exception as e:
        print(f"\n❌ Error during testing: {e}")
        import traceback
        traceback.print_exc()



# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_000_MissionOrchestratorAgent % python test_hitl_standalone.py
============================================================
Testing HITL Utilities
============================================================

1. Checking which tasks require approval...
     No approval needed: T1 - Collect customer information
   ✓ Requires approval: T2 - Verify documents
     No approval needed: T3 - Schedule onboarding call

2. Creating approval request...
   ✓ Created approval request:
     Task: T2 - Verify documents
     Agent: Document Verification Agent
     Status: pending

3. Processing approval decision...
   ✓ Approval processed:
     Task: T2
     Decision: approved
     Decided by: test_user

4. Checking approval status...
   ✓ Task T2 approved: True
   ✓ Task T1 approved: False (not in history)

5. Getting pending approvals...
   ✓ Found 0 pending approvals

6. Auto-approving for testing...
   ✓ Auto-approved 0 tasks

============================================================
HITL Utilities Tests Complete!
============================================================

============================================================
Testing Approval Check Node
============================================================

1. Testing with default config (auto_approve_for_testing=False)...
   ✓ Node executed successfully
   - Pending approvals: 1
   - Mission status: awaiting_approval
     ⚠ T2: Verify documents (requires approval)

2. Testing with auto-approval (simulated)...
   ✓ After auto-approval:
     - Pending approvals: 0
     - Approval history: 1 entries
     - Mission status: in_progress
       ✓ T2: approved by auto_approval

============================================================
Approval Check Node Test Complete!
============================================================

============================================================
Testing Full Flow with HITL Approval Workflow
============================================================

1-4. Setting up mission (goal → planning → data → ordering)...

5. Executing tasks with approval workflow...
   Task T1 completed
   Task T2 completed (requires approval)
     ⚠ 1 task(s) awaiting approval
       - T2: Verify documents
   Task T3 completed
     ⚠ 1 task(s) awaiting approval
       - T2: Verify documents

--- Mission Execution Summary ---
  Mission: Reduce Customer Onboarding Time
  Tasks completed: 3/3
  Mission status: awaiting_approval
  Pending approvals: 1
  Approval history: 0 entries

  Pending Approvals:
    ⚠ T2: Verify documents

============================================================
Full Flow with HITL Test Complete!
============================================================

✅ All tests completed successfully!
